In [1]:
from utils.jobshop_model_transport import FJTransportProblem
from utils.costant import *
from utils.utility import solve_all_and_gantt
from utils.utility import tune

In [2]:
problem = FJTransportProblem(symmetry_breaking=True)

In [3]:
problem.add_human()
problem.add_human()
problem.add_human()
problem.set_dur_hum(HUMAN_JOBS_TIME)
problem.add_robot()
problem.add_robot()
problem.set_dur_robot(ROBOT_JOBS_TIME)

id_kit_1 = problem.add_workstation(WS_KITTING)
id_kit_2 = problem.add_workstation(WS_KITTING)
id_grip = problem.add_workstation(WS_BUILDING_1)
id_gs = problem.add_workstation(WS_BUILDING_2)
id_pal = problem.add_workstation(WS_PALLETTING)
id_pal_2 = problem.add_workstation(WS_PALLETTING)

problem.add_transport(id_kit_1, id_grip)
problem.add_transport(id_kit_2,id_gs)
problem.add_transport(id_grip, id_gs)
problem.add_transport(id_gs, id_pal)

problem.set_t_conv(3)
problem.add_items_to_build(FLASHLIGHT_CLIPPED, 4)
problem.add_items_to_build(FLASHLIGHT_SCREWS, 4)

problem.make_ws_topology()


In [4]:
problem.model_problem(objective_type=0)

In [5]:
problem.solve(solver='ortools')

Solution found with makespan: 105
Computational time: 6.775 seconds


(True, 6.774856805801392, <ExitStatus.OPTIMAL: 2>)

In [6]:
problem.make_gantt()